# 사용준비

## 구글 드라이브 임포트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 작업 디렉토리 이동

In [2]:
cd /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total

/content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total


## DUST3R 필요 lib다운(설치후 재실행 해야함)

In [3]:
!pip install -r requirements.txt
!pip install -r requirements_optional.txt

## ZERO123++ 필요 lib다운

In [4]:
!pip install rembg
!pip install cog
!pip install diffusers
!pip install transformers
!pip install torch
!pip install accelerate
!pip install colmap-wrapper
!pip install PyOpenGL
!pip install PyOpenGL_accelerate
!pip install glfw

  Using cached cog-0.9.5-py3-none-any.whl (90 kB)
  Using cached fastapi-0.98.0-py3-none-any.whl (56 kB)
  Using cached pydantic-1.10.15-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached structlog-24.1.0-py3-none-any.whl (65 kB)
  Using cached starlette-0.27.0-py3-none-any.whl (66 kB)
  Using cached httptools-0.6.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (341 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.3 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.6.4
    Uninstalling pydantic-2.6.4:
      Successfully uninstalled pydantic-2.6.4
  Attempting uninstall: starlette
    Found existing installation: starlette 0.37.2
    Uninstalling starlette-0.37.2:
      Successfully uninstalled starlette-0.37.

## 확인중

In [ ]:
!sudo apt-get update
!sudo apt-get install -y colmap

# Zero123++ 사용코드

In [5]:
import os
import subprocess
from typing import List
import cv2
import rembg
import torch
from cog import BasePredictor, Input, Path
from diffusers import DiffusionPipeline, EulerAncestralDiscreteScheduler
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

class Predictor(BasePredictor):
    def setup(self) -> None:
        """Load the model into memory to make running multiple predictions efficient"""

        if not os.path.exists("weights"):
            os.mkdir("weights")


        print("Setting up pipeline...")

        self.pipeline = DiffusionPipeline.from_pretrained(
            "/content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/weights/zero123plusplus",
            custom_pipeline="/content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/diffusers-support/",
            torch_dtype=torch.float16,
            local_files_only=True
        )
        self.pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(
            self.pipeline.scheduler.config, timestep_spacing='trailing'
        )
        self.pipeline.to('cuda:0')

    def predict(
        self,
        image: Path = Input(description="Input image. Aspect ratio should be 1:1. Recommended resolution is >= 320x320 pixels."),
        remove_background: bool = Input(description="Remove the background of the input image", default=False),
        return_intermediate_images: bool = Input(description="Return the intermediate images together with the output images", default=False),
    ) -> List[Path]:
        """Run a single prediction on the model"""
        outputs = []

        cond = Image.open(str(image))
        ret, mask = cv2.threshold(np.array(cond.split()[-1]), 0, 255, cv2.THRESH_BINARY)
        x, y, w, h = cv2.boundingRect(mask)
        image_filename = "original" + image.suffix

        # optional background removal step
        if remove_background:
            rembg_session = rembg.new_session()
            cond = rembg.remove(cond, session=rembg_session)
            # image should be a png after background removal
            image_filename += ".png"

        if return_intermediate_images:
            temp_original = f"/tmp/{image_filename}"
            cond.save(temp_original)
            outputs.append(temp_original)

        all_results = self.pipeline(cond, num_inference_steps=100).images[0]
        side_len = all_results.width//2
        subimages = [all_results.crop((x, y, x + side_len, y+side_len)) for y in range(0, all_results.height, side_len) for x in range(0, all_results.width, side_len)]
        for i, output_img in enumerate(subimages):
            filename = f"/content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output{i+1}.png"
            output_img.save(filename)
            outputs.append(filename)

        return([Path(output) for output in outputs])


# Predictor 인스턴스 생성
predictor = Predictor()

# 모델 설정
predictor.setup()

# 이미지 변환 실행
result_paths = predictor.predict(image=Path("/content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Input/img006.jpg"), remove_background=True, return_intermediate_images=False)

# 결과 경로 출력
for path in result_paths:
    print(path)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Setting up pipeline...


Loading pipeline components...:   0%|          | 0/8 [00:00<?, ?it/s]

100%|████████████████████████████████████████| 176M/176M [00:00<00:00, 104GB/s]


  0%|          | 0/100 [00:00<?, ?it/s]

/content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output1.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output2.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output3.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output4.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output5.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output6.png


# Zero123++ 모델 사용 이미지들 전처리
  - 이미지 동일한 크기 및 비율 조정 (512 * 512)
  - 이미지 선명도 및 해상도


## 동일한 크기 및 비율 조정

In [ ]:
import cv2
import numpy as np

# 이미지 로드
image_paths = [f'/content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output{i}.png' for i in range(1, 7)]

# 목표 이미지 크기
target_width = 512
target_height = 512

# 각 이미지에 대해서 크기 및 비율 조정을 수행
for i, image_path in enumerate(image_paths):
    # 이미지를 읽어옴
    image = cv2.imread(image_path)

    # 이미지 크기 조정
    resized_image = cv2.resize(image, (target_width, target_height))

    # 결과 이미지 저장 경로
    resized_image_path = f'/content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output{i+1}.png'
    cv2.imwrite(resized_image_path, resized_image)

    # 결과 확인
    print(f'Image {i+1} resized to {target_width}x{target_height}, saved to:', resized_image_path)

Image 1 resized to 512x512, saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output1.png
Image 2 resized to 512x512, saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output2.png
Image 3 resized to 512x512, saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output3.png
Image 4 resized to 512x512, saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output4.png
Image 5 resized to 512x512, saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output5.png
Image 6 resized to 512x512, saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output6.png


##  이미지 샤프닝

In [ ]:
import cv2
import numpy as np

# 이미지 로드
image_paths = [f'/content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output{i}.png' for i in range(1, 7)]

# 각 이미지에 대해서 샤프닝을 수행
for i, image_path in enumerate(image_paths):
    # 이미지를 읽어옴
    image = cv2.imread(image_path)

    # 샤프닝을 위한 커널 정의
    kernel_sharpening = np.array([[-1, -1, -1],
                                  [-1,  9, -1],
                                  [-1, -1, -1]])

    # 이미지에 샤프닝 적용
    sharpened_image = cv2.filter2D(image, -1, kernel_sharpening)

    # 결과 이미지 저장 경로
    sharpened_image_path = f'/content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output{i+1}.png'
    cv2.imwrite(sharpened_image_path, sharpened_image)

    # 결과 확인
    print(f'Image {i+1} sharpened and saved to:', sharpened_image_path)


Image 1 sharpened and saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output1.png
Image 2 sharpened and saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output2.png
Image 3 sharpened and saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output3.png
Image 4 sharpened and saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output4.png
Image 5 sharpened and saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output5.png
Image 6 sharpened and saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output6.png


## 잡음 제거

In [ ]:
import cv2
import numpy as np

# 이미지 로드
image_paths = [f'/content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output{i}.png' for i in range(1, 7)]

# 각 이미지에 대해서 잡음 제거를 수행
for i, image_path in enumerate(image_paths):
    # 이미지를 읽어옴
    image = cv2.imread(image_path)

    # Gaussian Blur 적용
    # (5, 5)는 Gaussian Kernel의 크기, 0은 표준 편차를 자동으로 계산하게 함
    denoised_image = cv2.GaussianBlur(image, (5, 5), 0)

    # 결과 이미지 저장 경로
    denoised_image_path = f'/content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output{i+1}.png'
    cv2.imwrite(denoised_image_path, denoised_image)

    # 결과 확인
    print(f'Image {i+1} denoised and saved to:', denoised_image_path)


Image 1 denoised and saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output1.png
Image 2 denoised and saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output2.png
Image 3 denoised and saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output3.png
Image 4 denoised and saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output4.png
Image 5 denoised and saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output5.png
Image 6 denoised and saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output6.png


# DUSt3R 사용 코드

In [ ]:
#!/usr/bin/env python3

# 필요한 라이브러리 및 모듈을 임포트합니다.
import os
import torch
import numpy as np
import copy
import trimesh
from scipy.spatial.transform import Rotation

from dust3r.inference import inference, load_model
from dust3r.image_pairs import make_pairs
from dust3r.utils.image import load_images
from dust3r.utils.device import to_numpy
from dust3r.viz import add_scene_cam, CAM_COLORS, OPENGL, pts3d_to_trimesh, cat_meshes
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode

# 특정 GPU 설정을 위한 torch 설정입니다. (Ampere GPU와 PyTorch 1.12 이상 버전에서 사용)
torch.backends.cuda.matmul.allow_tf32 = True
batch_size = 1  # 배치 사이즈를 1로 설정합니다.

# 3D 장면 출력을 GLB 파일 형식으로 변환하는 함수입니다.
def _convert_scene_output_to_glb(outdir, imgs, pts3d, mask, focals, cams2world, cam_size=0.05,
                                 cam_color=None, as_pointcloud=False,
                                 transparent_cams=False):
    assert len(pts3d) == len(mask) <= len(imgs) <= len(cams2world) == len(focals)
    pts3d = to_numpy(pts3d)
    imgs = to_numpy(imgs)
    focals = to_numpy(focals)
    cams2world = to_numpy(cams2world)

    scene = trimesh.Scene()

    if as_pointcloud:
        pts = np.concatenate([p[m] for p, m in zip(pts3d, mask)])
        col = np.concatenate([p[m] for p, m in zip(imgs, mask)])
        pct = trimesh.PointCloud(pts.reshape(-1, 3), colors=col.reshape(-1, 3))
        scene.add_geometry(pct)
    else:
        meshes = []
        for i in range(len(imgs)):
            meshes.append(pts3d_to_trimesh(imgs[i], pts3d[i], mask[i]))
        mesh = trimesh.Trimesh(**cat_meshes(meshes))
        scene.add_geometry(mesh)

    # 카메라를 추가하는 코드 부분을 제거했습니다.

    rot = np.eye(4)
    rot[:3, :3] = Rotation.from_euler('y', np.deg2rad(180)).as_matrix()
    scene.apply_transform(np.linalg.inv(cams2world[0] @ OPENGL @ rot))
    outfile = os.path.join(outdir, 'scene.glb')
    scene.export(file_obj=outfile)
    return outfile


# 이미지들로부터 3D 모델을 처리하고 생성하는 메인 함수입니다.
def process_images_to_3d_model(input_image_paths, output_file_path, weights_path, device='cuda'):
    # 모델을 로드합니다.
    model = load_model(weights_path, device, verbose=False)

    # 입력 이미지들을 로드합니다.
    imgs = load_images(input_image_paths, size=512, verbose=False)

    # 이미지 쌍을 생성합니다.
    pairs = make_pairs(imgs, scene_graph='complete', prefilter=None, symmetrize=True)
    # 추론을 수행합니다.
    output = inference(pairs, model, device, batch_size=batch_size, verbose=False)

    # 글로벌 정렬을 수행하여 3D 장면을 생성합니다.
    scene = global_aligner(output, device=device, mode=GlobalAlignerMode.PointCloudOptimizer, verbose=False)
    loss = scene.compute_global_alignment(init='mst', niter=1000, schedule='linear', lr=0.01)

    # 필요한 후처리를 수행합니다.
    scene = scene.clean_pointcloud()
    scene = scene.mask_sky()

    rgbimg = scene.imgs
    focals = scene.get_focals().cpu()
    cams2world = scene.get_im_poses().cpu()
    pts3d = to_numpy(scene.get_pts3d())
    scene.min_conf_thr = float(scene.conf_trf(torch.tensor(3.0)))
    msk = to_numpy(scene.get_masks())

    # 최종적으로 GLB 파일을 생성합니다.
    _convert_scene_output_to_glb(os.path.dirname(output_file_path), rgbimg, pts3d, msk, focals, cams2world, as_pointcloud=False,
                                 transparent_cams=False, cam_size=0.05)

if __name__ == '__main__':

    # 입력 이미지 경로와 출력 파일 경로, 가중치 경로를 설정합니다.
    input_image_paths = []
    for i in range(1,4):
        input_image_paths.append(f"/content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output{i}.png")

    output_file_path = '/content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Output/output_model.glb'
    weights_path = '/content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth'

    # 이미지들로부터 3D 모델을 처리하고 생성합니다.
    process_images_to_3d_model(input_image_paths, output_file_path, weights_path)


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

# 원라인 코드

In [18]:
import os
import subprocess
from typing import List
import cv2
import rembg
import torch
from cog import BasePredictor, Input, Path
from diffusers import DiffusionPipeline, EulerAncestralDiscreteScheduler
from PIL import Image
import numpy as np
import cv2
import os
import torch
import trimesh
from scipy.spatial.transform import Rotation
from dust3r.inference import inference, load_model
from dust3r.image_pairs import make_pairs
from dust3r.utils.image import load_images
from dust3r.utils.device import to_numpy
from dust3r.viz import add_scene_cam, CAM_COLORS, OPENGL, pts3d_to_trimesh, cat_meshes
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode

# Zero123++ 모델 생성자
class Predictor(BasePredictor):
    def setup(self) -> None:
        """Load the model into memory to make running multiple predictions efficient"""

        if not os.path.exists("weights"):
            os.mkdir("weights")


        print("Setting up pipeline...")

        self.pipeline = DiffusionPipeline.from_pretrained(
            f"{os.getcwd()}/weights/zero123plusplus",
            custom_pipeline=f"{os.getcwd()}/diffusers-support/",
            torch_dtype=torch.float16,
            local_files_only=True
        )
        self.pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(
            self.pipeline.scheduler.config, timestep_spacing='trailing'
        )
        self.pipeline.to('cuda:0')

    def predict(
        self,
        image: Path = Input(description="Input image. Aspect ratio should be 1:1. Recommended resolution is >= 320x320 pixels."),
        remove_background: bool = Input(description="Remove the background of the input image", default=False),
        return_intermediate_images: bool = Input(description="Return the intermediate images together with the output images", default=False),
        Interim_deliverables: str = Input(),
        ) -> List[Path]:
        """Run a single prediction on the model"""
        outputs = []

        cond = Image.open(str(image))
        ret, mask = cv2.threshold(np.array(cond.split()[-1]), 0, 255, cv2.THRESH_BINARY)
        x, y, w, h = cv2.boundingRect(mask)
        image_filename = "original" + image.suffix

        # optional background removal step
        if remove_background:
            rembg_session = rembg.new_session()
            cond = rembg.remove(cond, session=rembg_session)
            # image should be a png after background removal
            image_filename += ".png"

        if return_intermediate_images:
            temp_original = f"/tmp/{image_filename}"
            cond.save(temp_original)
            outputs.append(temp_original)

        all_results = self.pipeline(cond, num_inference_steps=100).images[0]
        side_len = all_results.width//2
        subimages = [all_results.crop((x, y, x + side_len, y+side_len)) for y in range(0, all_results.height, side_len) for x in range(0, all_results.width, side_len)]
        for i, output_img in enumerate(subimages):
            filename = f"{Interim_deliverables}/output{i+1}.png"
            output_img.save(filename)
            outputs.append(filename)

        return([Path(output) for output in outputs])

# 이미지 크기 및 비율 맞추기
def Image_preprocessing(image_paths):

  # 목표 이미지 크기
  target_width = 512
  target_height = 512

  # 각 이미지에 대해서 크기 및 비율 조정을 수행
  for i, image_path in enumerate(image_paths):
      # 이미지를 읽어옴
      image = cv2.imread(image_path)

      # 샤프닝을 위한 커널 정의
      kernel_sharpening = np.array([[-1, -1, -1],
                                    [-1,  9, -1],
                                    [-1, -1, -1]])

      # 이미지 크기 조정
      resized_image = cv2.resize(image, (target_width, target_height))

      # 이미지에 샤프닝 적용
      sharpened_image = cv2.filter2D(resized_image, -1, kernel_sharpening)

      # Gaussian Blur 적용
      # (5, 5)는 Gaussian Kernel의 크기, 0은 표준 편차를 자동으로 계산하게 함
      denoised_image = cv2.GaussianBlur(sharpened_image, (5, 5), 0)

      cv2.imwrite(image_path, denoised_image)
      # 결과 확인
      print(f'Image {i+1} saved to:', image_path)

# 3D 장면 출력을 GLB 파일 형식으로 변환하는 함수입니다.
def _convert_scene_output_to_glb(outdir, imgs, pts3d, mask, focals, cams2world, cam_size=0.05,
                                 cam_color=None, as_pointcloud=False,
                                 transparent_cams=False):
    assert len(pts3d) == len(mask) <= len(imgs) <= len(cams2world) == len(focals)
    pts3d = to_numpy(pts3d)
    imgs = to_numpy(imgs)
    focals = to_numpy(focals)
    cams2world = to_numpy(cams2world)

    scene = trimesh.Scene()

    if as_pointcloud:
        pts = np.concatenate([p[m] for p, m in zip(pts3d, mask)])
        col = np.concatenate([p[m] for p, m in zip(imgs, mask)])
        pct = trimesh.PointCloud(pts.reshape(-1, 3), colors=col.reshape(-1, 3))
        scene.add_geometry(pct)
    else:
        meshes = []
        for i in range(len(imgs)):
            meshes.append(pts3d_to_trimesh(imgs[i], pts3d[i], mask[i]))
        mesh = trimesh.Trimesh(**cat_meshes(meshes))
        scene.add_geometry(mesh)

    rot = np.eye(4)
    rot[:3, :3] = Rotation.from_euler('y', np.deg2rad(180)).as_matrix()
    scene.apply_transform(np.linalg.inv(cams2world[0] @ OPENGL @ rot))
    outfile = os.path.join(outdir, 'scene.glb')
    scene.export(file_obj=outfile)
    return outfile


# 이미지들로부터 3D 모델을 처리하고 생성하는 메인 함수입니다.
def process_images_to_3d_model(input_image_paths, output_file_path, weights_path, device='cuda'):
    # 모델을 로드합니다.
    model = load_model(weights_path, device, verbose=False)

    # 입력 이미지들을 로드합니다.
    imgs = load_images(input_image_paths, size=512, verbose=False)

    # 이미지 쌍을 생성합니다.
    pairs = make_pairs(imgs, scene_graph='complete', prefilter=None, symmetrize=True)
    # 추론을 수행합니다.
    output = inference(pairs, model, device, batch_size=batch_size, verbose=False)

    # 글로벌 정렬을 수행하여 3D 장면을 생성합니다.
    scene = global_aligner(output, device=device, mode=GlobalAlignerMode.PointCloudOptimizer, verbose=False)
    loss = scene.compute_global_alignment(init='mst', niter=500, schedule='linear', lr=0.01)

    # 필요한 후처리를 수행합니다.
    scene = scene.clean_pointcloud()
    scene = scene.mask_sky()

    rgbimg = scene.imgs
    focals = scene.get_focals().cpu()
    cams2world = scene.get_im_poses().cpu()
    pts3d = to_numpy(scene.get_pts3d())
    scene.min_conf_thr = float(scene.conf_trf(torch.tensor(3.0)))
    msk = to_numpy(scene.get_masks())

    # 최종적으로 GLB 파일을 생성합니다.
    _convert_scene_output_to_glb(os.path.dirname(output_file_path), rgbimg, pts3d, msk, focals, cams2world, as_pointcloud=False,
                                 transparent_cams=False, cam_size=0.05)

def run(input_path, Interim_deliverables, output_path, weights_path):
  # Predictor 인스턴스 생성
  predictor = Predictor()

  # 모델 설정
  predictor.setup()

  # 이미지 변환 실행
  result_paths = predictor.predict(image=Path(input_path), remove_background=True, return_intermediate_images=False,Interim_deliverables=Interim_deliverables)

  # 경로 str화
  Interim_deliverables_paths = []
  for path in result_paths:
    Interim_deliverables_paths.append(str(path))

  # 생성된 이미지 전처리
  Image_preprocessing(Interim_deliverables_paths)

  # 특정 GPU 설정을 위한 torch 설정입니다.
  torch.backends.cuda.matmul.allow_tf32 = True
  batch_size = 1  # 배치 사이즈를 1로 설정합니다.

  # 이미지들로부터 3D 모델을 처리하고 생성합니다.
  process_images_to_3d_model(Interim_deliverables_paths, output_path, weights_path)

weights_path = f"{os.getcwd()}/checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth"
input = f"{os.getcwd()}/Data/Input/img006.jpg"
Interim_deliverables = f"{os.getcwd()}/Data/Interim_deliverables"
output_file_path = f"{os.getcwd()}/Data/Output/output_model.glb"
run(input, Interim_deliverables, output_file_path, weights_path)

Setting up pipeline...


Loading pipeline components...:   0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Image 1 saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output1.png
Image 2 saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output2.png
Image 3 saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output3.png
Image 4 saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output4.png
Image 5 saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output5.png
Image 6 saved to: /content/drive/MyDrive/TeamProjects/DimensionalPioneers/Total/Data/Interim_deliverables/output6.png


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

## 기존 dust3r 데모 코드(참고용)

In [ ]:
#!/usr/bin/env python3
# Copyright (C) 2024-present Naver Corporation. All rights reserved.
# Licensed under CC BY-NC-SA 4.0 (non-commercial use only).
#
# --------------------------------------------------------
# gradio demo
# --------------------------------------------------------
import argparse
import gradio
import os
import torch
import numpy as np
import tempfile
import functools
import trimesh
import copy
from scipy.spatial.transform import Rotation

from dust3r.inference import inference, load_model
from dust3r.image_pairs import make_pairs
from dust3r.utils.image import load_images, rgb
from dust3r.utils.device import to_numpy
from dust3r.viz import add_scene_cam, CAM_COLORS, OPENGL, pts3d_to_trimesh, cat_meshes
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode

import matplotlib.pyplot as pl
pl.ion()

torch.backends.cuda.matmul.allow_tf32 = True  # for gpu >= Ampere and pytorch >= 1.12
batch_size = 1


def get_args_parser():
    parser = argparse.ArgumentParser()
    parser_url = parser.add_mutually_exclusive_group()
    parser_url.add_argument("--local_network", action='store_true', default=False,
                            help="make app accessible on local network: address will be set to 0.0.0.0")
    parser_url.add_argument("--server_name", type=str, default=None, help="server url, default is 127.0.0.1")
    parser.add_argument("--image_size", type=int, default=512, choices=[512, 224], help="image size")
    parser.add_argument("--server_port", type=int, help=("will start gradio app on this port (if available). "
                                                         "If None, will search for an available port starting at 7860."),
                        default=None)
    parser.add_argument("--weights", type=str, required=True, help="path to the model weights")
    parser.add_argument("--device", type=str, default='cuda', help="pytorch device")
    parser.add_argument("--tmp_dir", type=str, default=None, help="value for tempfile.tempdir")
    parser.add_argument("--silent", action='store_true', default=False,
                        help="silence logs")
    return parser


def _convert_scene_output_to_glb(outdir, imgs, pts3d, mask, focals, cams2world, cam_size=0.05,
                                 cam_color=None, as_pointcloud=False,
                                 transparent_cams=False, silent=False):
    assert len(pts3d) == len(mask) <= len(imgs) <= len(cams2world) == len(focals)
    pts3d = to_numpy(pts3d)
    imgs = to_numpy(imgs)
    focals = to_numpy(focals)
    cams2world = to_numpy(cams2world)

    scene = trimesh.Scene()

    # full pointcloud
    if as_pointcloud:
        pts = np.concatenate([p[m] for p, m in zip(pts3d, mask)])
        col = np.concatenate([p[m] for p, m in zip(imgs, mask)])
        pct = trimesh.PointCloud(pts.reshape(-1, 3), colors=col.reshape(-1, 3))
        scene.add_geometry(pct)
    else:
        meshes = []
        for i in range(len(imgs)):
            meshes.append(pts3d_to_trimesh(imgs[i], pts3d[i], mask[i]))
        mesh = trimesh.Trimesh(**cat_meshes(meshes))
        scene.add_geometry(mesh)

    # add each camera
    for i, pose_c2w in enumerate(cams2world):
        if isinstance(cam_color, list):
            camera_edge_color = cam_color[i]
        else:
            camera_edge_color = cam_color or CAM_COLORS[i % len(CAM_COLORS)]
        add_scene_cam(scene, pose_c2w, camera_edge_color,
                      None if transparent_cams else imgs[i], focals[i],
                      imsize=imgs[i].shape[1::-1], screen_width=cam_size)

    rot = np.eye(4)
    rot[:3, :3] = Rotation.from_euler('y', np.deg2rad(180)).as_matrix()
    scene.apply_transform(np.linalg.inv(cams2world[0] @ OPENGL @ rot))
    outfile = os.path.join(outdir, 'scene.glb')
    if not silent:
        print('(exporting 3D scene to', outfile, ')')
    scene.export(file_obj=outfile)
    return outfile


def get_3D_model_from_scene(outdir, silent, scene, min_conf_thr=3, as_pointcloud=False, mask_sky=False,
                            clean_depth=False, transparent_cams=False, cam_size=0.05):
    """
    extract 3D_model (glb file) from a reconstructed scene
    """
    if scene is None:
        return None
    # post processes
    if clean_depth:
        scene = scene.clean_pointcloud()
    if mask_sky:
        scene = scene.mask_sky()

    # get optimized values from scene
    rgbimg = scene.imgs
    focals = scene.get_focals().cpu()
    cams2world = scene.get_im_poses().cpu()
    # 3D pointcloud from depthmap, poses and intrinsics
    pts3d = to_numpy(scene.get_pts3d())
    scene.min_conf_thr = float(scene.conf_trf(torch.tensor(min_conf_thr)))
    msk = to_numpy(scene.get_masks())
    return _convert_scene_output_to_glb(outdir, rgbimg, pts3d, msk, focals, cams2world, as_pointcloud=as_pointcloud,
                                        transparent_cams=transparent_cams, cam_size=cam_size, silent=silent)


def get_reconstructed_scene(outdir, model, device, silent, image_size, filelist, schedule, niter, min_conf_thr,
                            as_pointcloud, mask_sky, clean_depth, transparent_cams, cam_size,
                            scenegraph_type, winsize, refid):
    """
    from a list of images, run dust3r inference, global aligner.
    then run get_3D_model_from_scene
    """
    imgs = load_images(filelist, size=image_size, verbose=not silent)
    if len(imgs) == 1:
        imgs = [imgs[0], copy.deepcopy(imgs[0])]
        imgs[1]['idx'] = 1
    if scenegraph_type == "swin":
        scenegraph_type = scenegraph_type + "-" + str(winsize)
    elif scenegraph_type == "oneref":
        scenegraph_type = scenegraph_type + "-" + str(refid)

    pairs = make_pairs(imgs, scene_graph=scenegraph_type, prefilter=None, symmetrize=True)
    output = inference(pairs, model, device, batch_size=batch_size, verbose=not silent)

    mode = GlobalAlignerMode.PointCloudOptimizer if len(imgs) > 2 else GlobalAlignerMode.PairViewer
    scene = global_aligner(output, device=device, mode=mode, verbose=not silent)
    lr = 0.01

    if mode == GlobalAlignerMode.PointCloudOptimizer:
        loss = scene.compute_global_alignment(init='mst', niter=niter, schedule=schedule, lr=lr)

    outfile = get_3D_model_from_scene(outdir, silent, scene, min_conf_thr, as_pointcloud, mask_sky,
                                      clean_depth, transparent_cams, cam_size)

    # also return rgb, depth and confidence imgs
    # depth is normalized with the max value for all images
    # we apply the jet colormap on the confidence maps
    rgbimg = scene.imgs
    depths = to_numpy(scene.get_depthmaps())
    confs = to_numpy([c for c in scene.im_conf])
    cmap = pl.get_cmap('jet')
    depths_max = max([d.max() for d in depths])
    depths = [d/depths_max for d in depths]
    confs_max = max([d.max() for d in confs])
    confs = [cmap(d/confs_max) for d in confs]

    imgs = []
    for i in range(len(rgbimg)):
        imgs.append(rgbimg[i])
        imgs.append(rgb(depths[i]))
        imgs.append(rgb(confs[i]))

    return scene, outfile, imgs


def set_scenegraph_options(inputfiles, winsize, refid, scenegraph_type):
    num_files = len(inputfiles) if inputfiles is not None else 1
    max_winsize = max(1, (num_files - 1)//2)
    if scenegraph_type == "swin":
        winsize = gradio.Slider(label="Scene Graph: Window Size", value=max_winsize,
                                minimum=1, maximum=max_winsize, step=1, visible=True)
        refid = gradio.Slider(label="Scene Graph: Id", value=0, minimum=0,
                              maximum=num_files-1, step=1, visible=False)
    elif scenegraph_type == "oneref":
        winsize = gradio.Slider(label="Scene Graph: Window Size", value=max_winsize,
                                minimum=1, maximum=max_winsize, step=1, visible=False)
        refid = gradio.Slider(label="Scene Graph: Id", value=0, minimum=0,
                              maximum=num_files-1, step=1, visible=True)
    else:
        winsize = gradio.Slider(label="Scene Graph: Window Size", value=max_winsize,
                                minimum=1, maximum=max_winsize, step=1, visible=False)
        refid = gradio.Slider(label="Scene Graph: Id", value=0, minimum=0,
                              maximum=num_files-1, step=1, visible=False)
    return winsize, refid


def main_demo(tmpdirname, model, device, image_size, server_name, server_port, silent=False):
    recon_fun = functools.partial(get_reconstructed_scene, tmpdirname, model, device, silent, image_size)
    model_from_scene_fun = functools.partial(get_3D_model_from_scene, tmpdirname, silent)
    with gradio.Blocks(css=""".gradio-container {margin: 0 !important; min-width: 100%};""", title="DUSt3R Demo") as demo:
        # scene state is save so that you can change conf_thr, cam_size... without rerunning the inference
        scene = gradio.State(None)
        gradio.HTML('<h2 style="text-align: center;">DUSt3R Demo</h2>')
        with gradio.Column():
            inputfiles = gradio.File(file_count="multiple")
            with gradio.Row():
                schedule = gradio.Dropdown(["linear", "cosine"],
                                           value='linear', label="schedule", info="For global alignment!")
                niter = gradio.Number(value=300, precision=0, minimum=0, maximum=5000,
                                      label="num_iterations", info="For global alignment!")
                scenegraph_type = gradio.Dropdown(["complete", "swin", "oneref"],
                                                  value='complete', label="Scenegraph",
                                                  info="Define how to make pairs",
                                                  interactive=True)
                winsize = gradio.Slider(label="Scene Graph: Window Size", value=1,
                                        minimum=1, maximum=1, step=1, visible=False)
                refid = gradio.Slider(label="Scene Graph: Id", value=0, minimum=0, maximum=0, step=1, visible=False)

            run_btn = gradio.Button("Run")

            with gradio.Row():
                # adjust the confidence threshold
                min_conf_thr = gradio.Slider(label="min_conf_thr", value=3.0, minimum=1.0, maximum=20, step=0.1)
                # adjust the camera size in the output pointcloud
                cam_size = gradio.Slider(label="cam_size", value=0.05, minimum=0.001, maximum=0.1, step=0.001)
            with gradio.Row():
                as_pointcloud = gradio.Checkbox(value=False, label="As pointcloud")
                # two post process implemented
                mask_sky = gradio.Checkbox(value=False, label="Mask sky")
                clean_depth = gradio.Checkbox(value=True, label="Clean-up depthmaps")
                transparent_cams = gradio.Checkbox(value=False, label="Transparent cameras")

            outmodel = gradio.Model3D()
            outgallery = gradio.Gallery(label='rgb,depth,confidence', columns=3, height="100%")

            # events
            scenegraph_type.change(set_scenegraph_options,
                                   inputs=[inputfiles, winsize, refid, scenegraph_type],
                                   outputs=[winsize, refid])
            inputfiles.change(set_scenegraph_options,
                              inputs=[inputfiles, winsize, refid, scenegraph_type],
                              outputs=[winsize, refid])
            run_btn.click(fn=recon_fun,
                          inputs=[inputfiles, schedule, niter, min_conf_thr, as_pointcloud,
                                  mask_sky, clean_depth, transparent_cams, cam_size,
                                  scenegraph_type, winsize, refid],
                          outputs=[scene, outmodel, outgallery])
            min_conf_thr.release(fn=model_from_scene_fun,
                                 inputs=[scene, min_conf_thr, as_pointcloud, mask_sky,
                                         clean_depth, transparent_cams, cam_size],
                                 outputs=outmodel)
            cam_size.change(fn=model_from_scene_fun,
                            inputs=[scene, min_conf_thr, as_pointcloud, mask_sky,
                                    clean_depth, transparent_cams, cam_size],
                            outputs=outmodel)
            as_pointcloud.change(fn=model_from_scene_fun,
                                 inputs=[scene, min_conf_thr, as_pointcloud, mask_sky,
                                         clean_depth, transparent_cams, cam_size],
                                 outputs=outmodel)
            mask_sky.change(fn=model_from_scene_fun,
                            inputs=[scene, min_conf_thr, as_pointcloud, mask_sky,
                                    clean_depth, transparent_cams, cam_size],
                            outputs=outmodel)
            clean_depth.change(fn=model_from_scene_fun,
                               inputs=[scene, min_conf_thr, as_pointcloud, mask_sky,
                                       clean_depth, transparent_cams, cam_size],
                               outputs=outmodel)
            transparent_cams.change(model_from_scene_fun,
                                    inputs=[scene, min_conf_thr, as_pointcloud, mask_sky,
                                            clean_depth, transparent_cams, cam_size],
                                    outputs=outmodel)
    demo.launch(share=False, server_name=server_name, server_port=server_port)


if __name__ == '__main__':
    parser = get_args_parser()
    args = parser.parse_args()

    if args.tmp_dir is not None:
        tmp_path = args.tmp_dir
        os.makedirs(tmp_path, exist_ok=True)
        tempfile.tempdir = tmp_path

    if args.server_name is not None:
        server_name = args.server_name
    else:
        server_name = '0.0.0.0' if args.local_network else '127.0.0.1'

    model = load_model(args.weights, args.device, verbose=not args.silent)
    # dust3r will write the 3D model inside tmpdirname
    with tempfile.TemporaryDirectory(suffix='dust3r_gradio_demo') as tmpdirname:
        if not args.silent:
            print('Outputing stuff in', tmpdirname)
        main_demo(tmpdirname, model, args.device, args.image_size, server_name, args.server_port, silent=args.silent)


ImportError: cannot import name 'RootModel' from 'pydantic' (/usr/local/lib/python3.10/dist-packages/pydantic/__init__.cpython-310-x86_64-linux-gnu.so)

# colmap함수(폐기)

In [ ]:
# 환경 변수 설정 (Colab에서는 필요하지 않을 수 있음)
import os
os.environ['QT_QPA_PLATFORM'] = 'offscreen'

# 데이터베이스 파일과 이미지 디렉토리의 경로 설정
database_path = "/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/database.db"
image_path = "/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/Result"
output_path = "/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/output"

# 데이터 베이스 초기화
if os.path.exists(database_path):
    os.remove(database_path)

# 디렉토리 생성
!mkdir -p "{output_path}"

# 특징점 추출 (CPU 기반)
!colmap feature_extractor --database_path "{database_path}" --image_path "{image_path}" --SiftExtraction.use_gpu 0 --SiftExtraction.max_image_size 2000 --SiftExtraction.num_threads 4


# 특징점 매칭
!colmap exhaustive_matcher --database_path "{database_path}" --SiftMatching.use_gpu 0 --SiftMatching.max_ratio 0.9 --SiftMatching.max_distance 1

# 스파스 재구성
!colmap mapper --database_path "{database_path}" --image_path "{image_path}" --output_path "{output_path}"
